# Building Gradient Boosting Models

Clone the repo at:
www.github.com/numeristical/resources

Navigate to folder: "Building_GB_Models"

Open the notebook file: "BuildGBModel.ipynb"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, log_loss, r2_score
import xgboost as xgb
import seaborn as sns
import hyperopt as hp
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)

In [ ]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

## Ames Housing Data

We'll use the Ames housing data.  This is a small-ish (<3K) size data set of houses sold in Ames, Iowa from 2006 to 2010

In [ ]:
df_house = pd.read_csv('../GBIP/data/Ames_Housing_Data.tsv', delimiter='\t')
df_house = df_house.loc[df_house['Gr Liv Area']<=4000,:]
df_house['Garage Area'].fillna(0, inplace=True)
df_house.info()

In [ ]:
df_house.sample(5)

In [ ]:
# There are lots of features here...
df_house.columns

In [ ]:
# Let's focus on a smaller subset of features...
feat_1 = ['Lot Area','Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Gr Liv Area', 
        'Full Bath', 'Half Bath', 'Bedroom AbvGr',
         'Garage Area', 'Fireplaces']


In [ ]:
X = df_house.iloc[:,:-1]  # everything except Sale Price
y = df_house.SalePrice 

In [ ]:
X_train_full, X_test_full, y_train, y_test = train_test_split(X,y,test_size = 400, random_state=0)

In [ ]:
X.loc[:,feat_1].info()

## Model Building Process
- ### Explore and interrogate your data
- ### Start with a simple baseline model
- ### Use early stopping, tune `max_depth` first
- ### Check where your model is doing poorly
- ### Try to engineer features to address model shortcomings
    - #### Use domain expertise / common knowledge
- ### Save "big" hyperparameter tuning until the end


### Explore and interrogate the data
- Look at data before modeling!
    - Understand the distribution of the target variable (and know its mean).
    - Explore relationships between the target and the features
    - Look for "idiosyncracies" or other artifacts in your data
    - Try to understand what different values "mean"
    - Sometimes variables have deeper meanings....
<br>
- For regression problems, the `pairplot` in the seaborn package is a nice way to look at all of the bivariate relationships.  



In [ ]:
# Get the mean of your target variable
np.mean(df_house.SalePrice), np.min(df_house.SalePrice), np.max(df_house.SalePrice)

In [ ]:
# Make a histogram - doesn't look great at first
plt.hist(df_house.SalePrice);

In [ ]:
# Redo the histogram with "nicer" bins
binpts = np.linspace(0,700000, 70+1)
plt.hist(df_house.SalePrice, bins=binpts);

In [ ]:
# do a pairplot - this may take a minute or two to complete
sns.pairplot(df_house.loc[:,feat_1+['SalePrice']], plot_kws={'alpha':.1});

## Questions
- What variables seem to have the strongest relationship with the SalePrice?
- What is going on with 'Year Built' and 'Year Remod/Add'?

In [ ]:
# Lot Area is hard to see due to outliers - let's plot it again here
plt.scatter(df_house['Lot Area'], df_house['SalePrice'], alpha=.1)
plt.xlim([0,30000])

## Start with a simple baseline model
- Often people start modeling by assembling all the relevant features, trying automated feature selection and massive grid searches to get the best model.
- This typically results in a model with lots of features that are basically irrelevant.
- It is costly to have lots of features in your model!
    - Every feature is a potential failure point
- In most cases, can get most of the performance from relatively few features


In [ ]:
feat_0 = ['Lot Area',
 'Overall Qual',
 'Year Built',
 'Gr Liv Area',
 'Bedroom AbvGr']

In [ ]:
# create data sets of just the smaller subset of features
X_train_0 = X_train_full.loc[:, feat_0]
X_test_0 = X_test_full.loc[:, feat_0]

In [ ]:
# Create / train Random Forest on just 5 variables
rf0 = RandomForestRegressor(n_estimators=1000, n_jobs=-1)
rf0.fit(X_train_0, y_train)

In [ ]:
preds_rf0 = rf0.predict(X_test_0)

In [ ]:
# Get RMSE and MAE of the predictions
np.sqrt(mean_squared_error(y_test, preds_rf0)), mean_absolute_error(y_test, preds_rf0), r2_score(y_test, preds_rf0)

In [ ]:
# Plot predicted (y) vs actual (x)
plt.scatter(x=y_test, y=preds_rf0, alpha=.2, marker='.')
plt.plot([0,500000],[0,500000], 'k--')
plt.xlabel('Actual')
plt.ylabel('Predicted');

Now that we have a baseline of performance, we can make sure that the additional complexities (more features, GB instead of RF, etc.) are actually improving things.

Next, let's try the 11 feature model and see how much it improves our random forest.

In [ ]:
# create data sets of the set of 11 features
X_train_1 = X_train_full.loc[:, feat_1]
X_test_1 = X_test_full.loc[:, feat_1]

In [ ]:
# Create / train Random Forest on 11 variables
rf1 = RandomForestRegressor(n_estimators=1000, n_jobs=-1)
rf1.fit(X_train_1, y_train)

In [ ]:
preds_rf1 = rf1.predict(X_test_1)

In [ ]:
# Get RMSE and MAE of the 11-feature RF predictions
np.sqrt(mean_squared_error(y_test, preds_rf1)), mean_absolute_error(y_test, preds_rf1), r2_score(y_test, preds_rf1)

In [ ]:
# Compare to previous predictions
np.sqrt(mean_squared_error(y_test, preds_rf0)), mean_absolute_error(y_test, preds_rf0), r2_score(y_test, preds_rf0)

In [ ]:
np.sqrt(mean_squared_error(y_test, preds_rf1))/np.sqrt(mean_squared_error(y_test, preds_rf0))

This looks like a substantial improvement.  
- R2 from 87.3 -> 90.2.  
- 12% drop in RMSE

So we can feel confident the additional 6 variables are adding predictive power.

In [ ]:
# Plot predicted (y) vs actual (x)
plt.scatter(y_test, preds_rf1, alpha=.2, marker='.')
plt.plot([0,500000],[0,500000], 'k--')
plt.xlabel('Actual')
plt.ylabel('Predicted');

## Gradient Boosting Model Building
Now that we have a baseline to compare against, we can begin modeling with Gradient Boosting.  Having a baseline will help us determine the quality of our hyperparameter settings.

## XGBoost

For now, we will use the XGBoost package for Gradient Boosting.  XGBoost first came out in 2015.  The acronym is "eXtreme Gradient Boosting".  It implemented a number of theoretical and practical improvements, many of which have become standard in subsequent packages:

- *Native handling of missing data*: (Missing values are checked to see if they "fit" better on the left or right side of the tree for each split values)
- *Newton steps*: rather than just finding the gradient, XGBoost also looks at the second derivative to determine the step size.
- *Advanced Regularization*: XGBoost rederived the math to flexibly incorporate *shrinkage* (similar to Lasso/Ridge Regression) and penalize the number of splits.
- *Not checking every split*: XGBoost had a method for deriving quantiles to avoid checking every split (particularly in large datasets when a predictor has a large number of possible values).  Though not the default, this has since been shown to be effective both for speeding up the training *and* as another source of regularization
- *Distributed training*: XGBoost had implementations for working with large datasets that are not held in memory.

In [ ]:
# First let's use an XGBoost model with default parameters
xgb_def = xgb.XGBRegressor()
xgb_def

In [ ]:
xgb_def.fit(X_train_1, y_train)

In [ ]:
preds_xgb_def = xgb_def.predict(X_test_1)

In [ ]:
np.sqrt(mean_squared_error(y_test, preds_xgb_def)), mean_absolute_error(y_test, preds_xgb_def), r2_score(y_test, preds_xgb_def)

In [ ]:
# Baseline 11-feature RF predictions
np.sqrt(mean_squared_error(y_test, preds_rf1)), mean_absolute_error(y_test, preds_rf1), r2_score(y_test, preds_rf1)

- Note that we actually do **worse** than the Random Forest model when we use the default parameter settings.
- It is very difficult to get the number of trees / learning rate right without early stopping!


## Use Early Stopping!!

As we just saw, using the default parameters on gradient boosting can easily give you results that are not very good.  Here, it does worse than a simple Random Forest

However, this does NOT mean you need to do massive grid searches to get good results

As discussed in the Fundamentals of Gradient Boosting course:

- The three most important parameters in your boosting model are the *max_depth*, *learning_rate*, and *n_estimators*.  
- Setting these is made more challenging by the fact that they are highly interactive
- The best way to handle this is:
    - Set aside a validation set for early stopping
    - Use a low-ish `learning rate`
    - Use a high `n_estimators` (we will early stop)
    - Stop when performance on the validation set begins to degrade
    
XGBoost (like most other boosting packages) makes it easy to implement early stopping, as we demonstrate below:


In [ ]:
# Set a big number of trees, and a small learning rate
# We will not actually fit so many trees, since the early stopping will kick in
xgb1 = xgb.XGBRegressor(n_estimators=5000, learning_rate=.01)


In [ ]:
# Add trees until we show no improvement (i.e. no new low) for 10 trees
xgb1.fit(X_train_1, y_train, 
         eval_set=[(X_test_1, y_test)], 
        early_stopping_rounds = 10)

In [ ]:
preds_xgb1 = xgb1.predict(X_test_1)


In [ ]:
np.sqrt(mean_squared_error(y_test, preds_xgb1)), mean_absolute_error(y_test, preds_xgb1), r2_score(y_test, preds_xgb1)

In [ ]:
# Compared to last model
np.sqrt(mean_squared_error(y_test, preds_xgb_def)), mean_absolute_error(y_test, preds_xgb_def), r2_score(y_test, preds_xgb_def)

In [ ]:
# Baseline 11-feature RF predictions
np.sqrt(mean_squared_error(y_test, preds_rf1)), mean_absolute_error(y_test, preds_rf1), r2_score(y_test, preds_rf1)

With early stopping we get a significant improvement over the default GB model and a slight improvement over the baseline RF model.

In [ ]:
# Now let's tune the max depth

md_vals_vec=list(range(1,10))
rmse_vec = np.zeros(len(md_vals_vec))
for i,md in enumerate(md_vals_vec):
    print(f'Training with max_depth {md}')
    xgb_temp = xgb.XGBRegressor(max_depth=md, 
                        n_estimators=5000, learning_rate=.01, 
         early_stopping_rounds = 10) #, early_stopping_rounds=10)
    xgb_temp.fit(X_train_1, y_train, 
         eval_set=[(X_test_1, y_test)], 
                 verbose=0)
    preds = xgb_temp.predict(X_test_1)
    rmse_vec[i] = np.sqrt(mean_squared_error(y_test, preds))
    
## Note - for newer versions of xgboost, this might scream at you....


In [ ]:
# Plot performance vs. max_depth
plt.plot(md_vals_vec, rmse_vec, marker='x')
np.min(rmse_vec)

Best `max_depth` is at 3, let's try re-training at that max_depth value

In [ ]:
xgb2 = xgb.XGBRegressor(max_depth=3, n_estimators=5000, learning_rate=.01)


In [ ]:
# Add trees until we show no improvement (i.e. no new low) for 10 trees
xgb2.fit(X_train_1, y_train, 
         eval_set=[(X_test_1, y_test)], 
        early_stopping_rounds = 10)

In [ ]:
preds_xgb2 = xgb2.predict(X_test_1)


In [ ]:
np.sqrt(mean_squared_error(y_test, preds_xgb2)), mean_absolute_error(y_test, preds_xgb2), r2_score(y_test, preds_xgb2)

In [ ]:
# Compared to previous model (un-tuned max_depth)
np.sqrt(mean_squared_error(y_test, preds_xgb1)), mean_absolute_error(y_test, preds_xgb1), r2_score(y_test, preds_xgb1)

In [ ]:
np.sqrt(mean_squared_error(y_test, preds_xgb2))/np.sqrt(mean_squared_error(y_test, preds_xgb1))

We see a significant improvement in RMSE from tuning the `max_depth`

## Iterating and Improving the Model
Note that we are already iterating and improving our model!
<br>
We are following a cycle:
- Fit model
- Make predictions
- Evaluate the quality of our predictions
- Compare to previous models
- Make adjustments and repeat the cycle

However, thus far, we have just made simple "no-brainer" improvements - going from RF to GB, implementing early-stopping, tuning the max_depth.

Now, we start using our brains, our domain expertise to make further improvements.

## Failure Analysis
A great way to look for improvements is to examine the cases where the model made its biggest mistakes.

- Look at cases where the model performed poorly
- Try to understand why the model's prediction was so different from reality
- See if there is anything in the data that would reflect this reasoning
- Try to adjust model, add / engineer features to capture this useful information
- Rerun model and see if it improves!

In [ ]:
# We calculate the residuals - discrepancies between true answer and prediction
# Positive residual => model underpredicted true value
# Negative residual => model overpredicted true value
resids = (y_test - preds_xgb2)
abs_resids = np.abs(resids)

In [ ]:
plt.hist(resids, bins = np.linspace(-100000,100000,101));

In [ ]:
under_val_test = X_test_full.loc[resids>30000]
under_val_test

In [ ]:
over_val_test = X_test_full.loc[resids<-30000]
over_val_test

In [ ]:
feat_3 = ['Lot Area','Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Gr Liv Area', 
        'Full Bath', 'Half Bath', 'Bedroom AbvGr',
         'Garage Area', 'Fireplaces', 'BsmtFin SF 1']#, 'BsmtFin SF 2']

X_train_3 = X_train_full.loc[:, feat_3]
X_test_3 = X_test_full.loc[:, feat_3]

In [ ]:
xgb3 = xgb.XGBRegressor(max_depth=3, n_estimators=5000, learning_rate=.01)


In [ ]:
# Add trees until we show no improvement (i.e. no new low) for 10 trees
xgb3.fit(X_train_3, y_train, 
         eval_set=[(X_test_3, y_test)], 
        early_stopping_rounds = 10)

In [ ]:
preds_xgb3 = xgb3.predict(X_test_3)


In [ ]:
np.sqrt(mean_squared_error(y_test, preds_xgb3)), mean_absolute_error(y_test, preds_xgb3), r2_score(y_test, preds_xgb3)

In [ ]:
np.sqrt(mean_squared_error(y_test, preds_xgb2)), mean_absolute_error(y_test, preds_xgb2), r2_score(y_test, preds_xgb2)

In [ ]:
feat_3a = ['Lot Area','Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Gr Liv Area', 
        'Full Bath', 'Half Bath', 'Bedroom AbvGr',
         'Garage Area', 'Fireplaces', 'BsmtFin SF 1', 'BsmtFin SF 2']

X_train_3a = X_train_full.loc[:, feat_3a]
X_test_3a = X_test_full.loc[:, feat_3a]

In [ ]:
xgb3a = xgb.XGBRegressor(max_depth=3, n_estimators=5000, learning_rate=.01)


In [ ]:
# Add trees until we show no improvement (i.e. no new low) for 10 trees
xgb3a.fit(X_train_3a, y_train, 
         eval_set=[(X_test_3a, y_test)], 
        early_stopping_rounds = 10)

In [ ]:
preds_xgb3a = xgb3a.predict(X_test_3a)


In [ ]:
np.sqrt(mean_squared_error(y_test, preds_xgb3a)), mean_absolute_error(y_test, preds_xgb3a), r2_score(y_test, preds_xgb3a)

In [ ]:
np.sqrt(mean_squared_error(y_test, preds_xgb3)), mean_absolute_error(y_test, preds_xgb3), r2_score(y_test, preds_xgb3)

In [ ]:
X_train_full['BsmtFinSFtotal'] = X_train_full['BsmtFin SF 1']+X_train_full['BsmtFin SF 2']
X_test_full['BsmtFinSFtotal'] = X_test_full['BsmtFin SF 1']+X_test_full['BsmtFin SF 2']


In [ ]:
feat_4 = ['Lot Area','Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Gr Liv Area', 
        'Full Bath', 'Half Bath', 'Bedroom AbvGr',
         'Garage Area', 'Fireplaces', 'BsmtFinSFtotal']

X_train_4 = X_train_full.loc[:, feat_4]
X_test_4 = X_test_full.loc[:, feat_4]

In [ ]:
xgb4 = xgb.XGBRegressor(max_depth=3, n_estimators=5000, learning_rate=.01)


In [ ]:
# Add trees until we show no improvement (i.e. no new low) for 10 trees
xgb4.fit(X_train_4, y_train, 
         eval_set=[(X_test_4, y_test)], 
        early_stopping_rounds = 10)

In [ ]:
preds_xgb4 = xgb4.predict(X_test_4)


In [ ]:
np.sqrt(mean_squared_error(y_test, preds_xgb4)), mean_absolute_error(y_test, preds_xgb4), r2_score(y_test, preds_xgb4)

## Hyperparameter Optimization

### Practical Considerations
- How much is it worth to improve (metric) by x ? (From a business / practical point of view)
- How much time / computing do I want to spend on improvement?
- How often will I be re-training / re-implementing this model?
- What is the value of knowing / understanding this dataset / model really well?


## Setting Parameters in Gradient Boosting
There are several approaches to finding the best parameters for your Gradient Boosting Model
1. Do a massive "grid search"
2. Just play around manually
3. "Smart" parameter search that tries to search promising combinations

There are drawbacks to all of these:
- Grid search is extremely time consuming
- It is difficult to know if you are choosing appropriate parameters and ranges
- There may be other considerations than just metric performance (model size, coherence, training time)
- Manual approaches are haphazard, attention consuming
- Smart approaches are not always as smart as they could be

In [ ]:
# Now let's tune the max depth

md_vals_vec=list(range(1,10))
rmse_vec = np.zeros(len(md_vals_vec))
for i,md in enumerate(md_vals_vec):
    print(f'Training with max_depth {md}')
    xgb_temp = xgb.XGBRegressor(max_depth=md, 
                        n_estimators=5000, learning_rate=.01, 
         early_stopping_rounds = 10) #, early_stopping_rounds=10)
    xgb_temp.fit(X_train_4, y_train, 
         eval_set=[(X_test_4, y_test)], 
                 verbose=0)
    preds = xgb_temp.predict(X_test_4)
    rmse_vec[i] = np.sqrt(mean_squared_error(y_test, preds))
    
## Note - for newer versions of xgboost, this might scream at you....


In [ ]:
# Plot performance vs. max_depth
plt.plot(md_vals_vec, rmse_vec, marker='x')
np.min(rmse_vec)

## Let's manually explore some parameters one by one...

In [ ]:
param_vals_vec=[.7,.75,.8,.85,.9,.95,1]
rmse_vec = np.zeros(len(param_vals_vec))
for i,param_val in enumerate(param_vals_vec):
    xgb_temp = xgb.XGBRegressor(max_depth=4, 
                        n_estimators=10000, learning_rate=.01,
                        subsample=param_val, early_stopping_rounds = 10)
    xgb_temp.fit(X_train_4, y_train, 
         eval_set=[(X_test_4, y_test)], 
                 verbose=0)
    preds = xgb_temp.predict(X_test_4)
    rmse_vec[i] = np.sqrt(mean_squared_error(y_test, preds))

In [ ]:
plt.plot(param_vals_vec, rmse_vec)

In [ ]:

param_vals_vec=[.2,.3,.4,.5,.6,.7, .8,.9, 1]
rmse_vec = np.zeros(len(param_vals_vec))
for i,param_val in enumerate(param_vals_vec):
    xgb_temp = xgb.XGBRegressor(max_depth=4, 
                        n_estimators=10000, learning_rate=.01,
                        subsample=.75, colsample_bynode=param_val, early_stopping_rounds = 10)
    xgb_temp.fit(X_train_4, y_train, 
         eval_set=[(X_test_4, y_test)], 
                 verbose=0)
    preds = xgb_temp.predict(X_test_4)
    rmse_vec[i] = np.sqrt(mean_squared_error(y_test, preds))

In [ ]:
plt.plot(param_vals_vec, rmse_vec);

In [ ]:
np.min(rmse_vec)

In [ ]:

param_vals_vec=[.01,.1,1,2,3,5,10,20]
rmse_vec = np.zeros(len(param_vals_vec))
for i,param_val in enumerate(param_vals_vec):
    xgb_temp = xgb.XGBRegressor(max_depth=4, 
                        n_estimators=10000, learning_rate=.01,
                        subsample=.75, colsample_bynode=.9, reg_lambda=param_val, early_stopping_rounds = 10)
    xgb_temp.fit(X_train_4, y_train, 
         eval_set=[(X_test_4, y_test)], 
                 verbose=0)
    preds = xgb_temp.predict(X_test_4)
    rmse_vec[i] = np.sqrt(mean_squared_error(y_test, preds))

In [ ]:
plt.plot(param_vals_vec, rmse_vec);

In [ ]:
np.min(rmse_vec)

### Hyperopt Package
Idea: rather than try and exhaustively search a huge parameter space, focus the search on "promising" areas of the search space (based on what you have seen so far).

The `hyperopt` package can be a bit confusing and has some limitations, but overall does a reasonably good job of seraching the parameter space.

Let's see it in action

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

- First we must define the "loss function"
- This takes a set of parameters and output the loss value

In [ ]:
def eval_model(params):
    xgb_base = xgb.XGBRegressor()
    xgb_base.set_params(**params)
    xgb_base.set_params(**{'learning_rate':.01,
                            'n_estimators': 10000,
                            'early_stopping_rounds':20},)
    xgb_base.fit(X_train_4, y_train, eval_set=[(X_test_4, y_test)], 
                 verbose=False)
    preds = xgb_base.predict(X_test_4)
    return(np.sqrt(mean_squared_error(y_test, preds)))

- Next, we define the parameter space.  
- There are several options to define ranges.
    - `randint`: chooses a random integer between a lower and an upper bound
    - `choice`: choose from a set of specific values
    - `uniform`: choose a (float) in the range
    - `quniform`: choose a discrete range of floats
    
See more at hyperopt documentation:
http://hyperopt.github.io/hyperopt/getting-started/search_spaces/


In [ ]:
fspace1 = {
    'max_depth':hp.randint('max_depth', 2, 6), # Pythonic - means 2 through 5 inclusive
    'colsample_bynode': hp.quniform('colsample_bynode',.1,1,.1),
    'subsample': hp.quniform('subsample',.3,1,.05),
    'reg_lambda': hp.qloguniform('reg_lambda',np.log(.1),np.log(20),.1)
}

- Finally, we use the `fmin` function and record our trials in a `Trials` object
- you can also specify which algorithm to use (http://hyperopt.github.io/hyperopt/#algorithms) (`tpe.suggest` is "Tree of Parzen Estimators") 


In [ ]:
trials1 = Trials()
best = fmin(fn=eval_model, space=fspace1, algo=tpe.suggest, max_evals=100, trials=trials1)

In [ ]:
best

In [ ]:
eval_model(best)

In [ ]:
trials1.trials[:3]

In [ ]:
md_trial_vals = np.array([t['misc']['vals']['max_depth'] for t in trials1.trials])
csbn_trial_vals = np.array([t['misc']['vals']['colsample_bynode'] for t in trials1.trials])
ss_trial_vals = np.array([t['misc']['vals']['subsample'] for t in trials1.trials])
lambda_trial_vals = np.array([t['misc']['vals']['reg_lambda'] for t in trials1.trials])
loss_trial_vals = np.array([t['result']['loss'] for t in trials1.trials])
trial_nums = np.array([t['tid'] for t in trials1.trials])

In [ ]:
plt.scatter(trial_nums, loss_trial_vals, alpha=.4);

In [ ]:
plt.scatter(trial_nums, csbn_trial_vals, alpha=.2);

In [ ]:
plt.scatter(trial_nums, md_trial_vals, alpha=.2);

In [ ]:
plt.scatter(md_trial_vals, loss_trial_vals, alpha=.2);

In [ ]:
plt.scatter(csbn_trial_vals, loss_trial_vals, alpha=.2);

In [ ]:
plt.scatter(ss_trial_vals, loss_trial_vals, alpha=.2);

In [ ]:
plt.scatter(np.log(lambda_trial_vals), loss_trial_vals, alpha=.2);

In [ ]:
fspace2 = {
    'max_depth':hp.randint('max_depth', 4, 9), # Pythonic - means 4 through 7 inclusive
    'colsample_bynode': hp.quniform('colsample_bynode',.1,1,.1),
    'subsample': hp.quniform('subsample',.3,1,.05),
    'reg_lambda': hp.qloguniform('reg_lambda',np.log(.1),np.log(20),.1),

}

- Finally, we use the `fmin` function and record our trials in a `Trials` object
- you can also specify which algorithm to use (http://hyperopt.github.io/hyperopt/#algorithms) (`tpe.suggest` is "Tree of Parzen Estimators") 


In [ ]:
trials2 = Trials()
best2 = fmin(fn=eval_model, space=fspace2, algo=tpe.suggest, max_evals=200, trials=trials2)

In [ ]:
best2

In [ ]:
md_trial_vals = np.array([t['misc']['vals']['max_depth'] for t in trials2.trials])
csbn_trial_vals = np.array([t['misc']['vals']['colsample_bynode'] for t in trials2.trials])
ss_trial_vals = np.array([t['misc']['vals']['subsample'] for t in trials2.trials])
lambda_trial_vals = np.array([t['misc']['vals']['reg_lambda'] for t in trials2.trials])
loss_trial_vals = np.array([t['result']['loss'] for t in trials2.trials])
trial_nums = np.array([t['tid'] for t in trials2.trials])

In [ ]:
plt.scatter(trial_nums, loss_trial_vals, alpha=.4);

In [ ]:
plt.scatter(trial_nums, csbn_trial_vals, alpha=.2);

In [ ]:
plt.scatter(trial_nums, md_trial_vals, alpha=.2);

In [ ]:
plt.scatter(md_trial_vals, loss_trial_vals, alpha=.2);

In [ ]:
plt.scatter(csbn_trial_vals, loss_trial_vals, alpha=.2);

In [ ]:
plt.scatter(ss_trial_vals, loss_trial_vals, alpha=.2);

In [ ]:
plt.scatter(np.log10(lambda_trial_vals), loss_trial_vals, alpha=.2);

At this point, we can decide to use the parameter set that worked best.

In [ ]:
xgb_final = xgb.XGBRegressor()
xgb_final.set_params(**best2)
xgb_final.set_params(**{'learning_rate':.01,
                        'n_estimators': 10000,
                        'early_stopping_rounds':20},)

In [ ]:
xgb_final.fit(X_train_4, y_train, eval_set=[(X_test_4, y_test)], 
             verbose=False)
preds = xgb_final.predict(X_test_4)
np.sqrt(mean_squared_error(y_test, preds))

# Final comments on parameter search
- This example was a quite small dataset (~3000 data points), so we were able to run many iterations very quickly.  Typically these searches will take much longer, so it is more important to choose smart ranges.
- Note that lots of combinations gave very good results.  Typically if you get the `max_depth` right (using early stopping) and then pick "pretty good" values for the rest, you will be in good shape
- In practice, other considerations (e.g. model size, parsimony) may come into play.  Generally, you want to pick a "simpler" model if the difference in performance is negligible